# Time for change.

Somewhere close to the start of the English maths curriculum, pupils learn to work with coins, initially as physical manipulatives, then as icons and eventually as numbers. By the age of seven or eight, pupils are taught to “find different combinations of coins that equal the same amounts of money”. This sounds quite straightforward.

Finding the smallest number of coins to make a particular amount of money is something we do without consciously thinking about it, but if you had to teach a child to do this, what method would you suggest? It’s something which machines can do too: vending machines give change; good vending machines give change using the smallest number of coins they can. How would you program a vending machine to do that?

A greedy algorithm

It’s a problem that lends itself to a greedy algorithm:

* start with the largest value coin
* keep giving that coin until the amount left is less than the value of * the coin
* move on to the next largest value coin and do that again,
* keep moving on to the next largest value of coin until there are no more coin values left.

Following this algorithm by hand seems to give the answers we expect. E.g. 38p change: 20p, 10p, 5p, 2p, 1p.

A näive approach to coding this in Scratch might look a little like this:

In [4]:
total = int(input('how much money in pence? '))
coins = 0

while total >= 200:
    total -= 200
    coins += 1
while total >= 100:
    total -= 100
    coins += 1
while total >= 50:
    total -= 50
    coins += 1
while total >= 20:
    total -= 20
    coins += 1
while total >= 10:
    total -= 10
    coins += 1
while total >= 5:
    total -= 5
    coins += 1
while total >= 2:
    total -= 2
    coins += 1
while total >= 1:
    total -= 1
    coins += 1

print('You need', coins, 'coins')

how much money in pence?139
You need  6  coins


Well, it certainly seems to work, but no one is likely to claim this is a particularly elegant implementation - there's just so much repetition here, isn't there? And we've these horrible, hard-coded, magic numbers. 
Let's pass the coinage system as a list, and just iterate over the list. Here we go:

In [18]:
def numberofcoins1(amount, coins=[1, 2, 5, 10, 20, 50, 100, 200]):
    change = []
    while amount > 0 and coins != []:
        if amount >= coins[-1]:
            change = change + [coins[-1]]
            amount -= coins[-1]
        else:
            del coins[-1]
    return len(change)


amount = int(input('How much money in pence? '))
print('You need', numberofcoins1(amount), 'coins')

How much money in pence? 139
You need 6 coins


Same answers, and essentially the same method, but much easier to maintain, and indeed understand. 

This works well enough for English coins. We can generalise this though: it’s easy enough to modify it to work with US coinage too, with English pre-decimal (or post-Brexit?) currency, and with the equally Byzantine Galleons, Sickles and Knuts if you find yourself teaching programming at Hogwarts.

In [19]:
amount = int(input('How much money in cents? '))
print('You need', numberofcoins1(amount, [1, 5, 10, 25]), 'American coins')

How much money in cents? 39
You need 6 American coins


## Bit coins?

But look, it works well enough for this set of coins too: 128, 64, 32, 16, 8, 4, 2, 1. That is, the binary place values for eight bits.

In [14]:
amount = int(input('Give me a number in denary below 256: '))
print('That has', numberofcoins1(
    amount, [1, 2, 4, 8, 16, 32, 64, 128]), 'on bits')

Give me a number in denary below 256: 42
That has 3 on bits


Given the ease with which pupils can find the smallest number of coins for amounts, I think a set of, let’s call them, bit-coins, perhaps first as manipulatives, then as pictures, finally as numbers, could be the easiest way to get pupils converting numbers to binary.

The above Python code works well enough for working out the bit values, and could be used for other number bases too – use 64, 8 and 1 to get octal. Use 16 and 1 to get hexadecimal. I think a coin-based model such as this goes a long way to internalise an understanding of the maths here.

If we’d like to actually get the binary out, then a little remixing is needed.

In [24]:
def decimalToBinary(decimal):
    bitValues = [128, 64, 32, 16, 8, 4, 2, 1]
    bits = ""
    for value in bitValues:
        if decimal >= value:
            bits = bits + "1"
            decimal = decimal - value
        else:
            bits = bits + "0"
    return bits


amount = int(input('Give me a number in denary below 256: '))
print('In binary, that is ', decimalToBinary(amount))

Give me a number in denary below 256: 42
In binary, that is  00101010


## Alternative approaches

Turns out, we can do the whole thing in a single line of code, using integer division and remainders. I don't like this, but what do you think?

In [20]:
def numberofcoins0(amount):
    return amount // 200 + \
        amount % 200 // 100 + \
        amount % 200 % 100 // 50 + \
        amount % 200 % 100 % 50 // 20 + \
        amount % 200 % 100 % 50 % 20 // 10 + \
        amount % 200 % 100 % 50 % 20 % 10 // 5 + \
        amount % 200 % 100 % 50 % 20 % 10 % 5 // 2 + \
        amount % 200 % 100 % 50 % 20 % 10 % 5 % 2


amount = int(input('How much money in pence? '))
print('You need', numberofcoins0(amount), 'coins')

How much money in pence? 139
You need 6 coins


Functional programmers don't like iterating over lists: they'd rather keep calling a function with smaller and smaller lists of coins. Our base case here is that you can't make change if you don't have any coins.

In [23]:
def numberofcoins2(amount, coins=[1, 2, 5, 10, 20, 50, 100, 200], change=[]):
    if len(coins) == 0:
        return change
    elif amount >= coins[-1]:
        return numberofcoins2(amount - coins[-1], coins, change + [coins[-1]])
    else:
        return numberofcoins2(amount, coins[:-1], change)


amount = int(input('How much money in pence? '))
print('You need', len(numberofcoins2(amount)),
      'coins: ', numberofcoins2(amount))

How much money in pence? 139
You need 6 coins:  [100, 20, 10, 5, 2, 2]


## However...

All this looks great, doesn’t it? We’ve got a general algorithm here to solve all sorts of coin decomposition problems, and have generalised it to convert numbers from to binary and even to other number bases.

The problem is, this algorithm doesn’t always work.

What about coins worth 1p, 10p, 20p and 25p coins. How would you make 40p? If you follow the greedy algorithm, you’d use a 25p, 10p and five 1p coins, so that’s seven coins in all. If you stop and think a bit, you might decide that just two, 20p coins would be better! How come the greedy algorithm brakes down here?



In [25]:
amount = int(input('How much money in pence? '))
print('You need these coins:', numberofcoins2(amount, [1, 10, 20, 25]))

How much money in pence? 40
You need these coins:  [25, 10, 1, 1, 1, 1, 1]


Or, imagine that we only have 7p and 9p coins. What combination would you use to make 47p? My Python code happily returns five 9p coins, but that’s wrong! You can’t make 47p using just 7p and 9p ‘coins’. Why not? You can, however, make any amount bigger than 47p, although my greedy algorithm gets most of these wrong too.

In [26]:
amount = int(input('How much money in pence? '))
print('You need these coins:', numberofcoins2(amount, [7, 9]))

How much money in pence? 47
You need these coins:  [9, 9, 9, 9, 9]


Coming up with an algorithm that works for these situations too is a bit more tricky, but one approach relies on the recursive nature of the problem. Sticking with the English coinage, to make 38p, you could start with a 20p and make 18p, or start with 10p and make 28p, or start with 5p and make 33p, or start with 2p and make 36p, or start with 1p and make 37p. So the smallest number of ways to make 38p has to be one more than the smallest number of ways to make 18p, 28p, 33p, 36p or 37p, whichever is least.

Our recursive rule here looks like:
```
minimumcoins(x) = 1 + minimum (
	minimumcoins(x-200),
	minimumcoins(x-100),
	minumumcoins(x-50),
	minimumcoins(x-20),
	minimumcoins(x-10),
    minimumcoins(x-5),
    minimumcoins(x-2),
    minimumcoins(x-1))
```
With the base cases that minimumcoins for 200, 100, 50, 20, 10, 5, 2 and 1 are all 1, as the smallest number of coins needed to make 50p is pretty trivial!

Generalising this for any set of coin values gives us this recursive Python code:

In [38]:
import time


def numberofcoins3(amount, coins=[1, 2, 5, 10, 20, 50, 100, 200]):
    minCoins = amount
    if minCoins in coins:
        return 1
    else:
        for thisCoin in [coin for coin in coins if coin <= amount]:
            numCoins = 1 + numberofcoins3(amount - thisCoin, coins)
            if numCoins < minCoins:
                minCoins = numCoins
        return minCoins


amount = int(input('How much money in pence? '))
startTime = time.clock()
print('You need', numberofcoins3(amount), 'coins')
print(time.clock() - startTime, 'seconds')

How much money in pence? 35
You need 3 coins
4.538736999999998 seconds


There are things we can do to speed this up – using a dictionary to keep track of the minimum coins found so far for each amount we look up helps. If we go to the trouble of keeping track like this, it’s actually faster to build up the dictionary from the bottom up – with English coins, you need 1 coin to make 1p, 1 coin to make 2p, 2 coins to make 3p, 2 coins to make 4p, 1 coin to make 5p, and so on. To find out how many coins you need to make 6p, you need to take the minimum from making 5p (1), 4p (2) or 1p (1) and add on a coin to that, so that’s two coins, as you’d expect.

In [68]:
def numberofcoins4(amount, coins=[1, 2, 5, 10, 20, 50, 100, 200]):
    table = [float('inf')] * (amount + 1)
    table[0] = 0
    for value in range(1, amount + 1):
        for coin in coins:
            if coin <= value:
                withCoin = table[value - coin]
                if withCoin < float('inf') and withCoin + 1 < table[value]:
                    table[value] = withCoin + 1
    return table[amount]


amount = int(input('How much money in pence? '))
startTime = time.clock()
print('You need', numberofcoins4(amount), 'coins')
print(time.clock() - startTime, 'seconds')

How much money in pence? 39
You need 5 coins
0.001677999999998292 seconds


So, we’ve a simple maths problem that we cover in Year 2 maths lessons, but taking this into computing, we can automate solving this, we can generalise this to a far broader class of problems, we can link this to binary (and other number base) conversion, we can get to grips with some tricky logical bugs, and apply sophisticated techniques like recursion and memorisation. From this example alone, I reckon it’s well worth making these connections between the maths curriculum and computing!